# Neural Crest Cell Count Analysis
This code was used to analyze neural crest cell counts measured from section images.

Required inputs for this script:

1. .csv file documenting the neural crest cell counts (Snai2, Sox9) on experimental and contralateral control sides.

Script prepared by Mike Piacentino, May 2021

In [3]:
# Import data handling and analysis packages
import os
import glob
import pandas as pd
from scipy import stats

# Import plotting packages
import iqplot
import bokeh.io
from bokeh.io import output_file, show
from bokeh.layouts import column, row
bokeh.io.output_notebook()

Loading BokehJS ...

## Import Data

In [4]:
source_data = pd.read_csv('Fig2_source_data_cellcounts.csv')
source_data.head()

,Unnamed: 0,ROI,Count,Image,ExptDate,Treatment,Stains,Embryo,Somites,Section,Channel,EmbID
0,0,Cntl,31,20210511_pCAGdnBMPR1A_Snai2;RFP;Cad6B;BF_Emb3_...,20210511,pCAGdnBMPR1A,Snai2;RFP;Cad6B;BF,Emb3,8ss,sec1,Snai2,20210511_Emb3_Snai2
1,1,Expt,26,20210511_pCAGdnBMPR1A_Snai2;RFP;Cad6B;BF_Emb3_...,20210511,pCAGdnBMPR1A,Snai2;RFP;Cad6B;BF,Emb3,8ss,sec1,Snai2,20210511_Emb3_Snai2
2,0,Cntl,17,20181125_pCAGdnBMPR1AFLAG_NC11m3eGFPSnai2FLAG_...,20181125,pCAGdnBMPR1A,NC11m3eGFPSnai2FLAG,Emb2,6ss,sec3,Snai2,20181125_Emb2_Snai2
3,1,Expt,12,20181125_pCAGdnBMPR1AFLAG_NC11m3eGFPSnai2FLAG_...,20181125,pCAGdnBMPR1A,NC11m3eGFPSnai2FLAG,Emb2,6ss,sec3,Snai2,20181125_Emb2_Snai2
4,0,Cntl,15,20181125_pCAGdnBMPR1AFLAG_NC11m3eGFPSox9Pax7_E...,20181125,pCAGdnBMPR1A,NC11m3eGFPSox9Pax7,Emb6,8ss,sec2,Sox9,20181125_Emb6_Sox9


### Process data by generating the mean value of 2aRFP and dnBMPR1A cells per section, per embryo

In [11]:
# Get a list of treatments
treatment_list = source_data.Treatment.unique()
treatment_list = treatment_list.tolist()

# Group dataframe by treatment
mean_sections = (source_data.groupby(['Treatment', 'Channel', 'ExptDate', 'Embryo', 'Somites', 
                                       'EmbID','ROI'])['ROI','Count'].mean())
mean_sections.head()

Count
Treatment    Channel ExptDate Embryo Somites EmbID               ROI        
pCAGdnBMPR1A Snai2   20181125 Emb1   6ss     20181125_Emb1_Snai2 Cntl   18.0
                                                                 Expt   14.5
                              Emb2   6ss     20181125_Emb2_Snai2 Cntl   15.5
                                                                 Expt   15.0
                              Emb3   8ss     20181125_Emb3_Snai2 Cntl   22.0

## Plot and perform statistical analysis

In [12]:
################### Isolate data for analysis ###################
# Annotate data further to plot 
cntl_construct = '2aRFP'
expt_construct = 'dnBMPR1A'

# Pull out only cells and treaments of interest, and rename ROIs with the appropriate constructs
df=mean_sections.reset_index()
df.replace(to_replace = {'Cntl': cntl_construct, 'Expt': expt_construct}, inplace=True)

################### Plot as strip plot ###################
# Plot as strip plot
p1 = iqplot.strip(data=df
                ,q='Count', q_axis='y'
                ,cats=['Channel', 'ROI']
                ,y_range=(0,55)
                ,frame_height = 250, frame_width = 200
                ,y_axis_label= 'Average Cell Counts Per Section'
                ,x_axis_label='Treatment'
                ,palette = ['black', 'black', 'black', 'black']
                ,parcoord_column='EmbID'
              )

# p1.axis.axis_label_text_font_style = 'bold italic'
p1.axis.axis_label_text_font_size = '14px'
p1.axis.major_label_text_font_size = '12px'
p1.axis.axis_label_text_font_style = 'normal'
p1.xaxis.major_label_orientation = 7
show(row(p1))

################### Perform statistical analysis ###################

# Perform Paired t test for Snai2
snai2 = df.loc[df['Channel'] == 'SNAI2']
cntl_snai2 = snai2.loc[snai2['ROI'] == cntl_construct]['Count']
expt_snai2 = snai2.loc[snai2['ROI'] == expt_construct]['Count']
ttest_snai2 = stats.ttest_rel(cntl_snai2,expt_snai2)

# Display test results
print('Paired t-test results for Snai2 cell counts: \n\t\t statistic=' + str(ttest_snai2[0]) + 
    '\n\t\t p-value=' + str(ttest_snai2[1]))

# Perform Paired t test for Sox9
sox9 = df.loc[df['Channel'] == 'SOX9']
cntl_sox9 = sox9.loc[sox9['ROI'] == cntl_construct]['Count']
expt_sox9 = sox9.loc[sox9['ROI'] == expt_construct]['Count']
ttest_sox9 = stats.ttest_rel(cntl_sox9,expt_sox9)

# Display test results
print('Paired t-test results for Sox9 cell counts: \n\t\t statistic=' + str(ttest_sox9[0]) + 
    '\n\t\t p-value=' + str(ttest_sox9[1]))

Paired t-test results for Snai2 cell counts: 
		 statistic=nan
		 p-value=nan
Paired t-test results for Sox9 cell counts: 
		 statistic=nan
		 p-value=nan


## Normalize to control (dnBMPR1A count / 2aRFP count) and plot ratios

In [13]:
norm_df=df.filter(['EmbID', 'Channel', 'ROI', 'Count'])
norm_df=norm_df.pivot(index='EmbID', columns='ROI', values='Count')
norm_df=norm_df.reset_index()
norm_df['ExptDate'], norm_df['Emb'], norm_df['Channel'] = zip(*norm_df['EmbID'].map(lambda x: x.split('_')))
norm_df.replace(to_replace=['Snai2', 'Sox9'], value=['SNAI2', 'SOX9'], inplace=True)
norm_df['Expt/Cntl']=norm_df['dnBMPR1A']/norm_df['2aRFP']
norm_df=norm_df.filter(['EmbID', 'ExptDate', 'Emb', 'Channel', '2aRFP', 'dnBMPR1A', 'Expt/Cntl'])
norm_df.head()

ROI,EmbID,ExptDate,Emb,Channel,2aRFP,dnBMPR1A,Expt/Cntl
0,20181125_Emb1_Snai2,20181125,Emb1,SNAI2,18.000000,14.500000,0.805556
1,20181125_Emb1_Sox9,20181125,Emb1,SOX9,13.666667,13.333333,0.975610
2,20181125_Emb2_Snai2,20181125,Emb2,SNAI2,15.500000,15.000000,0.967742
3,20181125_Emb2_Sox9,20181125,Emb2,SOX9,12.333333,19.000000,1.540541
4,20181125_Emb3_Snai2,20181125,Emb3,SNAI2,22.000000,19.000000,0.863636


In [14]:
################### Plot as stripbox plot ###################
# Build Stripbox plot
stripbox = iqplot.stripbox(
                    # Data to plot
                        data=norm_df,
                        q='Expt/Cntl', q_axis='y',
                        cats='Channel', 

                    # Plot details
                        jitter=True, jitter_kwargs=dict(width=0.3),
                        marker_kwargs=dict(alpha=0.8, size=7
                                           ,color='dimgray'
                                          ),
                        box_kwargs=dict(line_color='black', line_width=1.5),
                        whisker_kwargs=dict(line_color='black', line_width=1.5),
                        median_kwargs=dict(line_color='black', line_width=2),
                        top_level='box',
                        frame_width=150, frame_height=350,

                    # Plot customizations
                        y_range=(0,1.8),
                        y_axis_label='Norm. Cell Counts (dnBMPR1A / 2a-RFP)',
                        x_axis_label='Marker',
                        show_legend=False,
)

# Final customizations
stripbox.axis.axis_label_text_font_size = '14px'
stripbox.axis.major_label_text_font_size = '12px'
stripbox.axis.axis_label_text_font_style = 'normal'


# View plot
show(stripbox)